# Flag integration cycles

Flag out intergration cycles that are less then a specified length of time

In [1]:
%matplotlib inline
import os
import pandas as pd
from astropy.time import Time
import astropy.units as u
from pymir import mirstr as m

In [2]:
def flag_cycle(s, selection='flagging.txt', delta=5):
    """`s` is a record from uvdump output
    
    delta {int} -- the time range to construct, centered on the `time` in s
    """
    t1 = (Time(s['time'], format='jd')-5*u.second).datetime.strftime('%y%b%d:%H:%M:%S')
    t2 = (Time(s['time'], format='jd')+5*u.second).datetime.strftime('%y%b%d:%H:%M:%S')
    
    select = f"time({t1},{t2})"
    
    with open(selection, 'a') as out_file:
        print(select, file=out_file)
    

def flag_inttime(uv, threshold=9, logfile='dump.txt', selection='flagging.txt', delete=False):
    uvdump = m(f"uvdump vis={uv} vars=inttime,ant1,ant2,time,source log={logfile}").run()
    print(uvdump)
    
    var = pd.read_csv(logfile, names=('inttime','ant1','ant2','time','source'))
    var = var[(var['ant1'].values==1)&(var['ant2'].values==2)] # Smaller list of selects
    
    var[var['inttime']<threshold].apply(flag_cycle, axis=1, selection=selection)
    
    uvflag = m(f"uvflag vis={uv} select=@{selection} flagval=flag").run()
    print(uvflag)
    
    if delete:
        os.remove(logfile)
        os.remove(selection)        

In [3]:
flag_inttime('Data/data1_test.uv/', threshold=9, delete=True)

uvdump vis=Data/data1_test.uv/ vars=inttime,ant1,ant2,time,source log=dump.txtUvdump: version 1.0 26-May-09
Records generated: 152984

uvflag vis=Data/data1_test.uv/ select=@flagging.txt flagval=flag
uvflag: Revision 1.8, 2014/09/30 14:23:39 UTC


Set flags to indicate bad data in visibility file Data/data1_test.uv/
All line channels and all wideband channels selected

Overview of flagging on visibility file Data/data1_test.uv/
Changed flags set to FALSE (data flagged as bad)
Total number of records selected: 39440; out of 152984 records
Antennas used: 1-6
Counts of correlations within selected channels
channel   Originally    Currently
Good:        65825356.           0.    Changed to bad:    65825356.
Bad:         14987204.    80812560.
wide      Originally    Currently
Good:               0.           0.    Changed to bad:           0.
Bad:                0.           0.

